In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer,Dense,Dropout

In [ ]:
data=pd.read_csv("thyroidDF.csv")

In [ ]:
data.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047


In [ ]:
data.shape

(9172, 31)

In [ ]:
data.isnull().sum()

age                       0
sex                     307
on_thyroxine              0
query_on_thyroxine        0
on_antithyroid_meds       0
sick                      0
pregnant                  0
thyroid_surgery           0
I131_treatment            0
query_hypothyroid         0
query_hyperthyroid        0
lithium                   0
goitre                    0
tumor                     0
hypopituitary             0
psych                     0
TSH_measured              0
TSH                     842
T3_measured               0
T3                     2604
TT4_measured              0
TT4                     442
T4U_measured              0
T4U                     809
FTI_measured              0
FTI                     802
TBG_measured              0
TBG                    8823
referral_source           0
target                    0
patient_id                0
dtype: int64

In [ ]:
data.drop(['TSH_measured','T3_measured','TT4_measured','T4U_measured','TBG_measured','referral_source','patient_id'])

KeyError: ignored

In [ ]:
#remapping target valueas ton diagnostic group
dignoses={'A':'hypothyroid conditions',
	  'B':'hypothyroid conditions',
	  'C':'hypothyroid conditions',
	  'D':'hypothyroid conditions',
	  'E':'hypothyroid conditions',
	  'F':'hypothyroid conditions',
	  'G':'hypothyroid conditions',
	  'H':'hypothyroid conditions',
	  'I':'binding protein',
	  'J':'binding protein',
	  'K':'binding protein',
	  'L':'replacement therapy',
	  'M':'replacement therapy',
	  'N':'replacement therapy',
	  'O':'antithyroid treatment',
	  'P':'antithyroid treatment',
	  'Q':'antithyroid treatment',
	  'R':'miscellaneous',
	  'S':'miscellaneous',
	  'T':'miscellaneous'}
data['target']=data['target'].map(diagnoses)#remapping

NameError: ignored

In [ ]:
data.dropna(subset=['target'],inplace=True)

In [ ]:
data['target'].value_counts()

-      6771
K       436
G       359
I       346
F       233
R       196
A       147
L       115
M       111
N       110
S        85
GK       49
AK       46
J        30
B        21
MK       16
Q        14
O        14
C|I      12
KJ       11
GI       10
H|K       8
D         8
FK        6
C         6
P         5
MI        2
LJ        1
GKJ       1
OI        1
D|R       1
E         1
Name: target, dtype: int64

In [ ]:
data[data.age>100]

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
2976,455,F,f,f,f,f,f,f,f,f,...,118.0,t,1.13,t,104.0,f,NaN,SVI,-,850530001
5710,65511,M,f,f,f,f,f,f,f,f,...,113.0,t,1.08,t,104.0,f,NaN,other,-,860210008
6392,65512,M,f,f,f,f,f,f,f,f,...,112.0,t,0.84,t,133.0,f,NaN,other,-,860403050
8105,65526,F,f,f,f,f,f,f,f,f,...,132.0,t,1.02,t,129.0,f,NaN,other,-,861014041


In [ ]:
#splitting the data values as x and y
x=data.iloc[:0:-1]
y=data.iloc[:,-1]

In [ ]:
x

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
9171,31,M,f,f,f,f,f,f,f,t,...,66.0,t,1.02,t,65.0,f,NaN,other,-,870119035
9170,47,F,f,f,f,f,f,f,f,f,...,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027
9169,69,M,f,f,f,f,f,f,f,f,...,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9168,22,M,f,f,f,f,f,f,f,f,...,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9167,56,M,f,f,f,f,f,f,f,f,...,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,60,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803048
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042


In [ ]:
y

0       840801013
1       840801014
2       840801042
3       840803046
4       840803047
          ...    
9167    870119022
9168    870119023
9169    870119025
9170    870119027
9171    870119035
Name: patient_id, Length: 9172, dtype: int64

In [ ]:
x['sex'].unique()

array(['M', 'F', nan], dtype=object)

In [ ]:
x['sex'].replace(np.nan,'F',inplace=True)

<ipython-input-27-720c3f96a604>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['sex'].replace(np.nan,'F',inplace=True)


In [ ]:
x['sex'].value_counts()

F    6379
M    2792
Name: sex, dtype: int64

In [ ]:
 #Converting the data type

x['age']=x['age'].astype('float')
x['TSH']=x['TSH'].astype('float')
x['T3']=x['T3'].astype('float')
x['TT4']=x['TT4'].astype('float')
x['T4U']=x['T4U'].astype('float')
x['FTI']=x['FTI'].astype('float') 
x['TBG']=x['TBG'].astype('float')

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9172 entries, 0 to 9171
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  9172 non-null   float64
 1   sex                  9172 non-null   object 
 2   on_thyroxine         9172 non-null   object 
 3   query_on_thyroxine   9172 non-null   object 
 4   on_antithyroid_meds  9172 non-null   object 
 5   sick                 9172 non-null   object 
 6   pregnant             9172 non-null   object 
 7   thyroid_surgery      9172 non-null   object 
 8   I131_treatment       9172 non-null   object 
 9   query_hypothyroid    9172 non-null   object 
 10  query_hyperthyroid   9172 non-null   object 
 11  lithium              9172 non-null   object 
 12  goitre               9172 non-null   object 
 13  tumor                9172 non-null   object 
 14  hypopituitary        9172 non-null   object 
 15  psych                9172 non-null   o

In [ ]:
#Encoding the categorical data 
#Encoding the independent (output)
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
#Categorical data


Ordinal_Encoder=OrdinalEncoder(dtype='int64')
x.iloc[:,1:16]=Ordinal_Encoder.fit_transform(x.iloc[:,1:16])
#Ordinal_Encoder.fit_transform(x[['sex']])

<ipython-input-31-d6dc6e57b5fd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.iloc[:,1:16]=Ordinal_Encoder.fit_transform(x.iloc[:,1:16])


In [ ]:
x

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
9171,31.0,1,0,0,0,0,0,0,0,1,...,66.0,t,1.02,t,65.0,f,NaN,other,-,870119035
9170,47.0,0,0,0,0,0,0,0,0,0,...,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027
9169,69.0,1,0,0,0,0,0,0,0,0,...,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9168,22.0,1,0,0,0,0,0,0,0,0,...,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9167,56.0,1,0,0,0,0,0,0,0,0,...,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,60.0,0,0,0,0,0,0,0,0,0,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803048
4,32.0,0,0,0,0,0,0,0,0,0,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
3,36.0,0,0,0,0,0,0,0,0,0,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
2,41.0,0,0,0,0,0,0,0,0,0,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042


In [ ]:
x.replace(np.nan,'0',inplace=True)

<ipython-input-34-be653e34d38c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.replace(np.nan,'0',inplace=True)


In [ ]:
x

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
9171,31.0,1,0,0,0,0,0,0,0,1,...,66.0,t,1.02,t,65.0,f,0,other,-,870119035
9170,47.0,0,0,0,0,0,0,0,0,0,...,75.0,t,0.85,t,88.0,f,0,other,-,870119027
9169,69.0,1,0,0,0,0,0,0,0,0,...,113.0,t,1.27,t,89.0,f,0,SVI,I,870119025
9168,22.0,1,0,0,0,0,0,0,0,0,...,91.0,t,0.92,t,99.0,f,0,SVI,-,870119023
9167,56.0,1,0,0,0,0,0,0,0,0,...,64.0,t,0.83,t,77.0,f,0,SVI,-,870119022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,60.0,0,0,0,0,0,0,0,0,0,...,0,f,0,f,0,t,26.0,other,-,840803048
4,32.0,0,0,0,0,0,0,0,0,0,...,0,f,0,f,0,t,36.0,other,S,840803047
3,36.0,0,0,0,0,0,0,0,0,0,...,0,f,0,f,0,t,26.0,other,-,840803046
2,41.0,0,0,0,0,0,0,0,0,0,...,0,f,0,f,0,t,11.0,other,-,840801042


In [ ]:
label_encoder=LabelEncoder()
y_dt=label_encoder.fit_transform(y)

In [ ]:
y=pd.DataFrame(y_dt, columns=['target'])

In [ ]:
y

0       840801013
1       840801014
2       840801042
3       840803046
4       840803047
          ...    
9167    870119022
9168    870119023
9169    870119025
9170    870119027
9171    870119035
Name: patient_id, Length: 9172, dtype: int64

In [ ]:
x=data.iloc[:,0:-1]
y=data.iloc[:,-1]

In [ ]:
x

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target
0,29,F,f,f,f,f,f,f,f,t,...,f,NaN,f,NaN,f,NaN,f,NaN,other,-
1,29,F,f,f,f,f,f,f,f,f,...,t,128.0,f,NaN,f,NaN,f,NaN,other,-
2,41,F,f,f,f,f,f,f,f,f,...,f,NaN,f,NaN,f,NaN,t,11.0,other,-
3,36,F,f,f,f,f,f,f,f,f,...,f,NaN,f,NaN,f,NaN,t,26.0,other,-
4,32,F,f,f,f,f,f,f,f,f,...,f,NaN,f,NaN,f,NaN,t,36.0,other,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,t,64.0,t,0.83,t,77.0,f,NaN,SVI,-
9168,22,M,f,f,f,f,f,f,f,f,...,t,91.0,t,0.92,t,99.0,f,NaN,SVI,-
9169,69,M,f,f,f,f,f,f,f,f,...,t,113.0,t,1.27,t,89.0,f,NaN,SVI,I
9170,47,F,f,f,f,f,f,f,f,f,...,t,75.0,t,0.85,t,88.0,f,NaN,other,-


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [ ]:
from imblearn.over_sampling import SMOTE
y_train.value_counts()

860730007    1
850920074    1
860910043    1
850527065    1
850612027    1
            ..
860819029    1
851018026    1
860305010    1
851224017    1
850513001    1
Name: patient_id, Length: 7337, dtype: int64

In [ ]:
os=SMOTE(random_state=0,k_neighbors=1)
x_bal,y_bal=os.fit_resample(x_train,y_train)
x_test_bal,y_test_bal=os.fit_resample(x_test,y_test)

ValueError: ignored

In [ ]:
from sklearn.preprocessing import StandardScalar
sc=StandardScalar()
x_bal=sc.fit_transform(x_bal)
x_test_bal=sc.transform(x_test_bal)

ImportError: ignored

In [ ]:
x_bal

NameError: ignored

In [ ]:
x_test_bal=pd.DataFrame(x_test_bal,columns=columns)

NameError: ignored

In [ ]:
x_bal=pd.DataFrame(x_bal,columns=columns)

NameError: ignored

In [ ]:
x_bal

NameError: ignored

In [ ]:
from sklearn.inspection import permutation_importance
results=permutation_importance(rtf,x_bal,y_bal,scoring='accuracy')

NameError: ignored